In [1]:
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import requests
from dotenv import load_dotenv
import os

# Dummy dataset 
### create dummy dataset of employee locations around Johor Bahru while their workplace is Hospital Sultanah Aminah (HSA)

In [ ]:
# Create a CloudScraper object
scraper = cloudscraper.create_scraper()

# Using URL of PropertyGuru rental property search to create dummy data for employee location around Johor Bahru
search_url = "https://www.propertyguru.com.my/property-for-rent?freetext=Johor+Bahru,+Johor&listing_type=rent&market=residential&district_code=JH016&region_code=MY01&search=true"

# Create list to store property information
location_list = []

# Create counter for the number of scraped properties
n_scraped = 0

# Loop through the search results pages
for page in range(1, 20):  # range(1, n_pages + 1):
    # Make a search request
    response = scraper.get(search_url + f"/{page}")

    # Parse the search results
    soup = BeautifulSoup(response.content, "lxml")

    # Detect captcha
    if "captcha" in soup.text:
        print("Captcha detected when trying to scrape " + search_url + f"/{page}")

    # Get list of property cards that contain property information (identified by div tags with a specific class)
    property_card_list = soup.find_all("div", {"class": "listing-card"})

    # Loop through the property cards
    for property_card in property_card_list:
        # Get property name (identified by an HTML anchor tag with a specific class and item property)
        name = property_card.find("a", {"class": "nav-link", "itemprop": "url"}).get_text()

        # Get property address (identified by an HTML span tag with a specific item property)
        address = property_card.find("span", {"itemprop": "streetAddress"}).get_text()

        # Add property to the property list
        location_list.append([name, address])
        
    # Update the number of scraped properties
    n_scraped += len(property_card_list)
    # Show number of scraped properties
    print(f"Number of scraped properties: {n_scraped}")

In [ ]:
# Show first 10 elements of the list of employee locations
print(location_list[:10])

# Show total number of employee locations
print(len(property_list))

# Convert list of employee locations to pandas dataframe
new_data = pd.DataFrame(property_list, columns=["name", "address"])

# create new csv file with new_data
new_data.to_csv("employee_commute_emissions.csv",index=False)

# Read the existing csv
df = pd.read_csv("employee_commute_emissions.csv")

# Append the new data to the existing dataframe
#df = pd.concat([df, new_data], ignore_index=True)

# Save dataframe as csv
#df.to_csv("employee_commute_emissions.csv", index=False)


In [7]:
# Read the existing csv
df = pd.read_csv("employee_commute_emissions.csv")

In [8]:
df.drop('Unnamed: 0',1,inplace= True)

<ipython-input-8-442f889ace59>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop('Unnamed: 0',1,inplace= True)


In [ ]:
df.info()
# if there is missing addresses, use Google Maps API to fill in missing addresses, else skip the the data enrichment step

# Data enrichment: Fill in missing addresses

In [ ]:
# Load environment variables from .env file
load_dotenv()

In [ ]:
# Get Google Maps API key from .env 
google_maps_api_key = os.getenv("google_maps_api_key")

In [14]:
# Create function to get missing address based on the property name via Google Maps API
def get_missing_address(row):
    # Check if address is missing
    if pd.isna(row["address"]): 
        # Base URL for the Google Maps Find Place API
        base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"

        # Parameters for the Find Place API request
        params = {
            "input": f"{row['name']}, Singapore",
            "inputtype": "textquery",
            "fields": "formatted_address",
            "key": google_maps_api_key
        }

        # Send Find Place API request and store the response
        response = requests.get(base_url, params=params)
        data = response.json()

        # Check if request was successful
        if data["status"] == "OK":
            # Extract address from the response
            address = data["candidates"][0]["formatted_address"]
        # If no address was found, give notification and use the original value (i.e. np.nan)
        else:
            print(f"No address found for {row['name']}")
            address = row["address"]
    # If an address is present, use that address
    else:
        address = row["address"]
    # Return address
    return address

In [ ]:
# Apply function to get missing addresses and store them in "address_new" column
# Cost: 2.18$. More precisely, 0.017$ per API call for 128 missing addresses.
# df["address_new"] = df.apply(get_missing_address, axis=1)

In [ ]:
# Save enriched dataframe as csv
# df.to_csv("data/preprocessing/rental_prices_jb_preprocessing_1.csv", index=False)


In [ ]:
# Load enriched data
df = pd.read_csv("data/preprocessing/rental_prices_jb_preprocessing_1.csv")

In [ ]:
# Show dataframe info
df.info()

In [ ]:
# Percent missing addresses after 
print(f'Percent missing addresses after: {100 * pd.isna(df["address_new"]).sum() / pd.isna(df["address_new"]).count():.1f}%

In [ ]:
# Delete the remaining 23 missing addresses
df.dropna(subset=["address_new"], inplace=True)

# Using Google Maps API to calculate distance to workplace of all addresses

In [10]:

# Create function to get latitude and longitude from an address 
def get_latitude_longitude(address):
    # Base URL for the Google Maps Geocoding API
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    
    # Parameters for the Geocoding API request
    params = {
        "address": f"{address}, Malaysia",
        "key": google_maps_api_key
    }
    
    # Send Geocoding API request and store the response
    response = requests.get(base_url, params=params)
    data = response.json()
    
    # Check if request was successful
    if data["status"] == "OK":
        # Extract latitude and longitude from the response
        location = data["results"][0]["geometry"]["location"]
        latitude = location["lat"]
        longitude = location["lng"]
    else:
        # Assign missing values and print error message if the request failed
        latitude = np.nan
        longitude = np.nan
        print(f"Geocoding request failed for {address}")
    
    # Return latitude and longitude
    return (latitude, longitude)

In [ ]:
# Apply function to create latitude and longitude column 
# Cost: 8.41$. More precisely, 0.005$ per API call for 1682 addresses.
# df[["latitude", "longitude"]] = df["address_new"].apply(get_latitude_longitude).apply(pd.Series)

In [12]:
df[["latitude", "longitude"]] = df["address"].apply(get_latitude_longitude).apply(pd.Series)

In [13]:
df

,name,address,latitude,longitude
0,R&F Princess Cove,"Jalan Sultan Ibrahim, Off Lebuhraya Sultan Isk...",1.460929,103.770057
1,Puteri Habour Teega Residences,"Puteri Habour Teega, Iskandar Puteri (Nusajaya...",1.414982,103.653338
2,Parc Regency Serviced Apartment,"Jalan Masai Jaya, Off Jalan Masai Baru, Plento...",1.522762,103.812292
3,The Country Residency,"Jalan Rimba, Bandar Seri Alam, Johor Bahru, Johor",1.513081,103.882636
4,R&F Princess Cove,"Jalan Sultan Ibrahim, Off Lebuhraya Sultan Isk...",1.460929,103.770057
...,...,...,...,...
394,R&F Princess Cove,"Jalan Sultan Ibrahim, Off Lebuhraya Sultan Isk...",1.460929,103.770057
395,R&F Princess Cove,"Jalan Sultan Ibrahim, Off Lebuhraya Sultan Isk...",1.460929,103.770057
396,R&F Princess Cove,"Jalan Sultan Ibrahim, Off Lebuhraya Sultan Isk...",1.460929,103.770057
397,Setia Sky 88,"Jalan Dato Abdullah Tahir, Johor Bahru, Johor ...",1.478863,103.760337


In [ ]:
# Save enriched dataframe as csv
# df.to_csv("data/preprocessing/rental_prices_singapore_preprocessing_2.csv", index=False)

In [15]:
df.to_csv("data/preprocessing/employee_commute_emissions_2.csv", index=False, start =1)

In [20]:
# Create function to get meters to work place (Hospital Sultanah Aminah) 
def get_meters_to_hsa(row):
    # Get latitude and longitude of the property
    emploc_latitude = row["latitude"]
    emploc_longitude = row["longitude"]
    
    # Return a missing value if latitude or longitude is missing
    if np.isnan(emploc_latitude) or np.isnan(emploc_longitude):
        print(f"Employee location latitude or longitude missing for {row['address']}")
        return np.nan
    
    # Latitude and longitude of work place location (ie. Hospital Sultanah Aminah  )
    HSA_latitude = 1.458699
    HSA_longitude = 103.746044

    # Base URL for the Google Maps Distance Matrix API
    base_url = "https://maps.googleapis.com/maps/api/distancematrix/json"
    
    # Parameters for the Distance Matrix API request
    params = {
        "origins": f"{emploc_latitude},{emploc_longitude}",
        "destinations": f"{HSA_latitude},{HSA_longitude}",
        "key": google_maps_api_key
    }

    # Send the Distance Matrix API request and store the response
    response = requests.get(base_url, params=params)
    data = response.json()

    # Process the response to get the distance
    if "rows" in data and data["rows"]:
        meters_to_hsa = data["rows"][0]["elements"][0]["distance"]["value"]
        print(f"Distance between employee location and HSA: {meters_to_hsa} meters")
    else:
        print("No distance information available.")
        return np.nan
    return meters_to_hsa

In [ ]:
# Apply function to create the "meters_to_cbd" column 
# Cost: 8.4$. More precisely, 0.005$ per distance for 1680 distances.
# df["meters_to_cbd"] = df.apply(get_meters_to_cbd, axis=1)

In [ ]:
# Save enriched dataframe as csv
# df.to_csv("data/preprocessing/rental_prices_singapore_preprocessing_3.csv", index=False)

In [ ]:
df.apply(get_meters_to_hsa, axis=1)

In [22]:
# Show dataframe info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 0 to 398
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   name                 399 non-null    object 
 1   address              399 non-null    object 
 2   latitude             399 non-null    float64
 3   longitude            399 non-null    float64
 4   meters_to_workplace  399 non-null    int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 15.7+ KB


In [23]:
df

,name,address,latitude,longitude,meters_to_workplace
0,R&F Princess Cove,"Jalan Sultan Ibrahim, Off Lebuhraya Sultan Isk...",1.460929,103.770057,7969
1,Puteri Habour Teega Residences,"Puteri Habour Teega, Iskandar Puteri (Nusajaya...",1.414982,103.653338,22171
2,Parc Regency Serviced Apartment,"Jalan Masai Jaya, Off Jalan Masai Baru, Plento...",1.522762,103.812292,18782
3,The Country Residency,"Jalan Rimba, Bandar Seri Alam, Johor Bahru, Johor",1.513081,103.882636,22274
4,R&F Princess Cove,"Jalan Sultan Ibrahim, Off Lebuhraya Sultan Isk...",1.460929,103.770057,7969
...,...,...,...,...,...
394,R&F Princess Cove,"Jalan Sultan Ibrahim, Off Lebuhraya Sultan Isk...",1.460929,103.770057,7969
395,R&F Princess Cove,"Jalan Sultan Ibrahim, Off Lebuhraya Sultan Isk...",1.460929,103.770057,7969
396,R&F Princess Cove,"Jalan Sultan Ibrahim, Off Lebuhraya Sultan Isk...",1.460929,103.770057,7969
397,Setia Sky 88,"Jalan Dato Abdullah Tahir, Johor Bahru, Johor ...",1.478863,103.760337,4848


In [24]:
df.to_csv("data/preprocessing/employee_commute_emissions_3.csv", index=False)

In [3]:
# Load enriched data
df = pd.read_csv("data/preprocessing/employee_commute_emissions_3.csv")

In [6]:
df.index = df.index +1

In [12]:
df["employee_ID"]= df.index

In [13]:
df

,name,address,latitude,longitude,meters_to_workplace,employee_ID
1,R&F Princess Cove,"Jalan Sultan Ibrahim, Off Lebuhraya Sultan Isk...",1.460929,103.770057,7969,1
2,Puteri Habour Teega Residences,"Puteri Habour Teega, Iskandar Puteri (Nusajaya...",1.414982,103.653338,22171,2
3,Parc Regency Serviced Apartment,"Jalan Masai Jaya, Off Jalan Masai Baru, Plento...",1.522762,103.812292,18782,3
4,The Country Residency,"Jalan Rimba, Bandar Seri Alam, Johor Bahru, Johor",1.513081,103.882636,22274,4
5,R&F Princess Cove,"Jalan Sultan Ibrahim, Off Lebuhraya Sultan Isk...",1.460929,103.770057,7969,5
...,...,...,...,...,...,...
395,R&F Princess Cove,"Jalan Sultan Ibrahim, Off Lebuhraya Sultan Isk...",1.460929,103.770057,7969,395
396,R&F Princess Cove,"Jalan Sultan Ibrahim, Off Lebuhraya Sultan Isk...",1.460929,103.770057,7969,396
397,R&F Princess Cove,"Jalan Sultan Ibrahim, Off Lebuhraya Sultan Isk...",1.460929,103.770057,7969,397
398,Setia Sky 88,"Jalan Dato Abdullah Tahir, Johor Bahru, Johor ...",1.478863,103.760337,4848,398


In [ ]:
# Create function to get 
def get_employee_CO2(meters_to_workplace):
    CO2_emissions_from_employee_travel = 

In [ ]:
df["meters_to_workplace"].apply(get_latitude_longitude).apply(pd.Series)

In [ ]:
# There are a total of 260 working days in the 2023 calendar year
total_distance_travelled = meters_to_workplace

In [19]:
df["employee_ID"] = 'EmpID00' + df.index.astype(str)

In [ ]:
# In realworld data, collect this information from the employees. For the dummy dataset, I created a random list

In [20]:
df["transportation_to_work"]

,name,address,latitude,longitude,meters_to_workplace,employee_ID
1,R&F Princess Cove,"Jalan Sultan Ibrahim, Off Lebuhraya Sultan Isk...",1.460929,103.770057,7969,EmpID001
2,Puteri Habour Teega Residences,"Puteri Habour Teega, Iskandar Puteri (Nusajaya...",1.414982,103.653338,22171,EmpID002
3,Parc Regency Serviced Apartment,"Jalan Masai Jaya, Off Jalan Masai Baru, Plento...",1.522762,103.812292,18782,EmpID003
4,The Country Residency,"Jalan Rimba, Bandar Seri Alam, Johor Bahru, Johor",1.513081,103.882636,22274,EmpID004
5,R&F Princess Cove,"Jalan Sultan Ibrahim, Off Lebuhraya Sultan Isk...",1.460929,103.770057,7969,EmpID005


In [ ]:

# CO2 Emissions according to UK goverment emmission factor
#https://www.gov.uk/government/publications/greenhouse-gas-reporting-conversion-factors-2022
dict_co2e = dict(zip(['Small_car' ,'Medium_car', 'Large_car', 'Average_car','Small_motorbike' ,'Medium_motorbike', 'Large_motorbike', 'Average_motorbike'], [.14652, .18470, .27639, .17048, .08306, .10090, .13245, .11355]))